# Initialize
Load problem data, install packages

In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# !pip install deap
# !git clone https://github.com/jekim526/IE-541-Project.git
# !git --git-dir=/content/IE-541-Project/.git pull

import pandas as pd
import numpy as np
import sys
sys.path.append("..")
sys.path.append('/content/IE-541-Project')
import knapsack_EA_functions as ea

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
sys.path.append('/content/IE-541-Project')
import knapsack_EA_functions as ea

In [4]:
url = 'https://raw.githubusercontent.com/jekim526/IE-541-Project/main/data/Z_r_100_25_1.csv' # Get the "Raw" link
df1_o = pd.read_csv(url)
df1 = pd.DataFrame(df1_o).to_numpy() # DataFrame to nummpy array
n_size = len(df1) - 1

# Array of item value
item_value = [0] * n_size
for x in range(0,n_size):
  item_value[x] = df1[1,x+1]

# Array of item weight
item_weight = [0] * n_size
for x in range(0,n_size):
  item_weight[x] = df1[0,x+1]

# n x n array of joint profit
joint_profit = np.zeros((n_size,n_size))
for x in range(0, n_size-1):
  for y in range(0, n_size):
    joint_profit[x,y] = df1[x+2,y+1]

item_weight = np.array(item_weight)
item_value = np.array(item_value)

## Prepare the GA input and solve the instance via GA

In [8]:
# -------  NOTES: ------------------------------------------------------------------------------
# instance_setings should be a tuple contains:
# {item_value, item_weight, joint_profit, capacities}
#                 capacities = {capacity1, capacity2, ...}
#
# evolution_general_parameters should be a tuple contains:
# {popsize, swap_prob, mute_prob, punish_factor}, in which:
#    swap_prob is independent probability for swap at each point in uniform crossover.
#    mute_prob is independent probability for each attribute to be flipped in flip-bit mutation.
#
# evolution_specify_parameters should be a tuple contains: {CXPB, MUTPB},In which:
#    CXPB is the probability with which two individuals
#          are crossed
#    MUTPB is the probability for mutating an individual
#    MAX_GEN is the maximum generation threshold
#    STOP_GEN is the threshold of no progress generations

''' the maximize covalue_case '''
max_c = 0; max_c = max(max_c,max(np.sum(joint_profit,axis = 1))); max_c = max(max_c,max(np.sum(joint_profit,axis = 0)))
max_i = max(item_value)

''' prepare this solver inputs: '''
#initialize evolution parameter setting
evolution_general_parameters = (100, 0.1, 0.1, max_i + max_c)
evolution_specify_parameters = (0.5, 0.25, 1000, 100)
num_of_knapsack = 3
capacity = (sum(item_weight)/num_of_knapsack)*0.8 # 80% of the sum of all item weights divided by the number of knapsacks
capacities = (capacity,)*num_of_knapsack
#initialize instance setting
instance_settings = (item_value, item_weight, joint_profit, capacities)

In [9]:
''' solve via GA in one line! '''
objf_type = 2
#objf_type = (0.5,0.5,0)
#objf_type: 
# 1: the first objective function (maximum the total profit)
# 2: maximum the negative of total_weight
# 3: maximum the profit of the package with minmum of profit

best_ind,pop,num_gen = ea.perform_GA_base(objf_type, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)
best_ind.fitness.values
best_ind

C:\Users\Karlz\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Karlz\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Individual([[0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [1, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [1, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0],
           

In [10]:
def total_weight(decision_matrix: np.ndarray,  # n*m (m:number of knapsack, n: number of items)
                 item_value: np.ndarray,  # n,
                 item_weight: np.ndarray,  # n,
                 joint_profit: np.ndarray,  # n*n
                 ) -> float:
    total_weight = 0
    item_weight = item_value.reshape(len(item_weight), 1)
    for k in range(decision_matrix.shape[1]):
        kth_decision = decision_matrix[:, k]
        total_weight += np.dot(kth_decision, item_weight)
    return -total_weight
total_weight(best_ind, item_value,item_weight,joint_profit)

Individual([0], dtype=int64)

In [18]:
print(np.dot(best_ind[:,0],item_weight))
print(np.dot(best_ind[:,1],item_weight))
print(np.dot(best_ind[:,2],item_weight))

265
553
403


In [21]:
decision_matrix = best_ind
total_weight = 0
item_weight = item_value.reshape(len(item_weight), 1)
for k in range(decision_matrix.shape[1]):
    kth_decision = decision_matrix[:, k]
    total_weight += np.dot(kth_decision, item_weight)
total_weight

Individual([0], dtype=int64)

In [24]:
item_weight

array([[ 0],
       [ 0],
       [ 0],
       [57],
       [60],
       [52],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [98],
       [70],
       [88],
       [ 0],
       [84],
       [ 0],
       [ 0],
       [68],
       [ 0],
       [ 0],
       [ 0],
       [92],
       [ 0],
       [ 0],
       [94],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [86],
       [26],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [25],
       [78],
       [54],
       [24],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [38],
       [37],
       [88],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [24],
       [ 0],
       [27],
       [ 0],
       [87],
       [ 0],
       [ 0],

In [22]:
import numpy
import random
def random_complete(ind, instance_settings):
    item_weight = instance_settings[1]
    capacities = instance_settings[3]
    rcapcity = capacities - numpy.dot(ind.T, item_weight)

    mylist = numpy.where(numpy.sum(best_ind, axis=1) == 0)[0]
    random.shuffle(mylist)

    num_knapsack = len(ind[0])
    knapsack_list = list(range(num_knapsack))
    random.shuffle(knapsack_list)
    for i in mylist:
        for k in knapsack_list:
            if item_weight[i] < rcapcity[k]:
                ind[i] = numpy.zeros(num_knapsack)
                ind[i, k] = 1
                rcapcity[k] -= item_weight[i]
                break
    return ind
pp1o = pop[5].copy()
pp1 = pop[5].copy()
print(capacities - numpy.dot(pp1.T, item_weight))
pp1 = random_complete(pp1, instance_settings)
print(capacities - numpy.dot(pp1.T, item_weight))
# # np.where(pp1o != pp2o)
np.where(pp1o != pp1)

[-22.46666667  60.53333333   1.53333333]


NameError: name 'random' is not defined

In [370]:
def exam_feasibility(individual, instance_settings):
    if (numpy.sum(individual,axis=1) > 1).any():
        raise RuntimeError('Serious Issue: Item in Multi Knapsack')
    result = []
    item_weight = instance_settings[1]
    capacities = instance_settings[3]
    for k in range(individual.shape[1]):

        kth_knapsack = individual[:, k]
        weight = numpy.dot(kth_knapsack, item_weight)
        if weight > capacities[k]:
            result.append(False)
        else:
            result.append(True)
    return result
pp1[0] = [1,1,0]
pp1
exam_feasibility(pp1, instance_settings)

RuntimeError: Serious Issue: Item in Multi Knapsack

In [72]:
from deap import base
import objfuncs as objf
import operators as op
ind1 = best_ind
indo = ind1.copy()
''' initialize population '''
toolbox = base.Toolbox()
toolbox.register("evaluate", objf.objf_weight, objc_weight_vector=(0.5,0,0.5), instance_settings=instance_settings,
                 punish_factor=-1000)
ee1 = list(map(toolbox.evaluate, [ind1]))
print(ee1)

def mutLocalSearch(ind, item_weight, capacities, toolbox):
    rcapcity = capacities - numpy.dot(ind.T, item_weight)
    curr_obj = list(map(toolbox.evaluate, [ind]))[0][0]
    item_num = ind.shape[0]
    knapsack_num = ind.shape[1]
    change_pair = [0] * op.get_combination_num(knapsack_num,2)
    kk = 0
    for i in range(knapsack_num):
        for j in range(i, knapsack_num):
            if j == i:
                continue
            change_pair[kk] = (i, j)
            kk += 1
    # If all are zero, no need to exchange
    for i in range(item_num):
        if sum(ind[i]) == 0:
            continue
        for m, n in change_pair:
            if ind[i, m] == 0 and ind[i, n] == 0:
                continue
            # If it makes the instance infeasible no need to exchange
            change_m = (- ind[i, m] + ind[i, n]) * item_weight[i]
            change_n = (- ind[i, n] + ind[i, m]) * item_weight[i]
            if rcapcity[m] + change_m < 0:
                continue
            if rcapcity[n] + change_n < 0:
                continue
            # if it not increases the joint_value, (maybe) no need to exchange
            # ....
            # Now try exchange, see if it increases the ObjValue
            ind[i, m], ind[i, n] = ind[i, n], ind[i, m]
            new_obj = list(map(toolbox.evaluate, [ind]))[0][0]
            if curr_obj > new_obj:  # did not increase the ObjValue, change back
                ind[i, m], ind[i, n] = ind[i, n], ind[i, m]
                continue
            curr_obj = new_obj
            rcapcity[m] -= change_m
            rcapcity[n] -= change_n
    return ind
nind = mutLocalSearch(ind1, item_weight, capacities, toolbox)
print(list(map(toolbox.evaluate, [nind])))

[[13478.0]]
[[13478.0]]


In [63]:
knapsack_num = 3
change_pair = [0] * op.get_combination_num(knapsack_num,2)
kk = 0
for i in range(knapsack_num):
    for j in range(i, knapsack_num):
        if j == i:
            continue
        change_pair[kk] = (i, j)
        kk += 1
change_pair

[(0, 1), (0, 2), (1, 2)]

In [49]:
list(map(toolbox.evaluate, ind1))

IndexError: tuple index out of range

In [11]:
l = [1, 2, 3]
l1 = []
for i in l:
    print(l)
    l1.append(i + 10)
def inc(x):
    return x + 10
print(list(map(inc, l)))

[1, 2, 3]
[1, 2, 3]
[1, 2, 3]
[11, 12, 13]


In [66]:
import random

mylist = np.where(np.sum(best_ind,axis=1) == 0)[0]
random.shuffle(mylist)

print(mylist[0:max(len(mylist,10))])


TypeError: len() takes exactly one argument (2 given)

In [75]:
print(best_ind[0])
best_ind[0,2] = 1
print(best_ind[0])

[0 0 1]
[0 0 1]


In [80]:
aa = np.zeros((2,3))
aa
aa[1][1]=2
aa

array([[0., 0., 0.],
       [0., 2., 0.]])

In [5]:
evolution_general_parameters = (100, 0.1, 0.1, 200)
evolution_specify_parameters = (0.5, 0.25, 200, 100)
best_ind,pop,num_gen = ea.perform_GA_base(1, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)
best_ind.fitness.values

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


(15025.0,)

In [8]:
# def exam_feasibility(individual, instance_settings):
#   result = []
#   item_weight = instance_settings[1]
#   capacities = instance_settings[3]
#   for k in range(individual.shape[1]):
#     kth_knapsack = individual[:,k]
#     weight = np.dot(kth_knapsack, item_weight)
#     if weight > capacities[k]:
#       result.append(False) ## exceed the weight limitation, infeasible
#     else:
#       result.append(True) ## feasible
#   return result
#   # return True means feasible, 
ea.exam_feasibility(best_ind, instance_settings)    

[True, True, True, True, True]

Since above naive GA amazingly always produce feasibile solution, I want to verify my implementation of `exam_feasibility`. The exam below shows my implementation of `exam_feasibility` is correct.

In [ ]:
def exam_feasibility(individual, item_weight, capacities):
  result = []
  for k in range(individual.shape[1]):
    kth_knapsack = individual[:,k]
    weight = np.dot(kth_knapsack, item_weight)
    if weight > capacities[k]:
      result.append(False) ## exceed the weight limitation, infeasible
    else:
      result.append(True) ## feasible
  return result
  # return True means feasible, 

ind = np.array([[0,0,0],[0,1,0],[1,0,1]]).T # Note: in individual, every column is a decision vector of 
weight = np.array([20,30,40])
caps = (30,30,30)
exam_feasibility(ind,weight,caps)

In [ ]:
#Output the best individual as .csv file
import pandas as pd
df_best_ind = pd.DataFrame(best_ind.T)
df_best_ind.to_csv("/content/best_ind.csv")